# Projet 5 DataV2 BottleNeck

## Notebook 2 : Nettoyage des données

Ce notebook créé un fichier csv pour pouvoir faire les analyses. 

[lien vers le notebook 1 : analyse préparatoire](Barrios_Mathieu_1_notebook_072022.ipynb)<br>

[lien vers le notebook 3 : analyse demandée](analyse.ipynb)<br>

## code python
### importation des bibliothèques

In [1]:
import warnings # pour gerer les alarmes de openyxl (inclus dans pandas)
import pandas as pd
from IPython.display import display

#fonction perso se trouvant dans le dossier fonction (plus d'info à la fin du notebook)
import fonction.fonction_perso as perso 


### Importation des données

In [2]:
warnings.simplefilter("ignore")# openpyxl me met un warning car le systeme ne reconnait pas l'extension excel 

erp = pd.read_excel('donnee/erp.xlsx')
vente_web = pd.read_excel('donnee/web.xlsx')
liaison = pd.read_excel('donnee/liaison.xlsx')

### Nettoyage des données
#### Nettoyage du fichier ERP
affichage du DataFrame apres nettoyage

In [3]:
#netttoyage des donnees du fichier ERP

#drop des colonnes inutiles pour analyse
erp = erp[['product_id','price','onsale_web','stock_quantity']]

#affichage général du dataframe
display(erp.head())

#voir si une clé est possible avec ce dataframe
perso.detection_clé(erp)

,product_id,price,onsale_web,stock_quantity
0,3847,24.2,1,0
1,3849,34.3,1,0
2,3850,20.8,1,0
3,4032,14.1,1,0
4,4039,46.0,1,0


la colonne product_id est une clé


#### Nettoyage du fichier vente_web

In [4]:
#drop des colonnes inutiles
vente_web = vente_web[['sku','total_sales','post_name']]
#nettoyage des doublons
vente_web = vente_web.drop_duplicates()
#affichage du dataFrame
display(vente_web.head())
#detection si clé possible dans ce dataframe
perso.detection_clé(vente_web)

,sku,total_sales,post_name
0,bon-cadeau-25-euros,10.0,bon-cadeau-de-25-euros
1,15298,6.0,pierre-jean-villa-saint-joseph-preface-2018
2,15296,0.0,pierre-jean-villa-saint-joseph-tilde-2017
3,15300,0.0,pierre-jean-villa-croze-hermitage-accroche-coe...
4,19814,3.0,pierre-jean-villa-igp-gamine-2018


la colonne post_name est une clé


##### la colonne sku n'est pas une clé valide, verification des  doublons dans sku

In [5]:
#affichage des double dans la colonne sku
print("doublons dans la colonne sku")
display(vente_web[vente_web['sku'].duplicated()])
print("NaN dans la colonne sku")
display(vente_web[vente_web['sku'].isna()])

doublons dans la colonne sku


,sku,total_sales,post_name
470,NaN,0.0,pierre-jean-villa-cote-rotie-fongeant-2017
471,NaN,0.0,pierre-jean-villa-condrieu-suspendu-2018


NaN dans la colonne sku


,sku,total_sales,post_name
178,NaN,NaN,NaN
470,NaN,0.0,pierre-jean-villa-cote-rotie-fongeant-2017
471,NaN,0.0,pierre-jean-villa-condrieu-suspendu-2018


##### enlever les lignes fautives et vérification que l'on n'enlève que 2 lignes

In [6]:
#elimination et verification que l'on supprime que le nombre de lignes necessaires
print("nombre de ligne avant drop na :",vente_web.shape[0])
vente_web = vente_web[~vente_web['sku'].isna()] #elimination des valeurs NA
print("nombre de ligne après drop na :",vente_web.shape[0],"\n")

#détection de clé possible après nettoyage:
perso.detection_clé(vente_web)


nombre de ligne avant drop na : 717
nombre de ligne après drop na : 714 

la colonne sku est une clé
la colonne post_name est une clé


### Nettoyage du fichier liaison
changement du nom de colonne id_web -> sku

In [7]:
#changement nom de colonne (indiqué dans le mail de Sylvie)
liaison = liaison.rename(columns={'id_web' : 'sku'})

### Jointure pour créer le fichier d'analyse 

In [8]:
# jointure du fichier

join_liaison_erp = pd.merge(erp,liaison,on =('product_id'),how='outer',indicator = True)

print("vérification s'il n'y aurait pas des ref manquantes entre liaison et erp")
display(join_liaison_erp[join_liaison_erp['_merge']!='both'])

#enlever la colonne _merge car verif OK
join_liaison_erp = join_liaison_erp.drop(columns='_merge')

#jointure de la deuxieme partie

#jointure totale pour une BDD avec toutes les references des magasins (NaN si pas en web)
join_erp_venteweb = pd.merge(join_liaison_erp,vente_web,on='sku',how='outer',indicator = True)


print("Vérification si ref manquantes entre liaison et vente web")
         
display(join_erp_venteweb[join_erp_venteweb['_merge']!='both'])
print("C'est normal si onsale_web = 0 le fichier n'est pas en vente sur le net")
#enlever la colonne merge
join_erp_venteweb = join_erp_venteweb.drop(columns='_merge')

print("verification si tous les NaN dans SKu sont bien dans les on_saleweb 0")
display(join_erp_venteweb[((join_erp_venteweb['onsale_web']!=0) 
                   & (join_erp_venteweb['sku']=="NaN"))])  
print ("Vérification doublons dans la base finale")
display(join_erp_venteweb[join_erp_venteweb.duplicated()])

#mettre des post_name_vide à la place des Nan dans la colonne post_name
join_erp_venteweb['post_name']=join_erp_venteweb['post_name'].fillna('post_name_vide')

vérification s'il n'y aurait pas des ref manquantes entre liaison et erp


,product_id,price,onsale_web,stock_quantity,sku,_merge


Vérification si ref manquantes entre liaison et vente web


,product_id,price,onsale_web,stock_quantity,sku,total_sales,post_name,_merge
19,4055,86.1,0,0,NaN,NaN,NaN,left_only
20,4090,73.0,0,0,NaN,NaN,NaN,left_only
21,4092,47.0,0,0,NaN,NaN,NaN,left_only
22,4195,14.1,0,0,NaN,NaN,NaN,left_only
23,4209,73.5,0,0,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...
717,5954,18.8,0,0,15609,NaN,NaN,left_only
718,5955,27.3,0,0,14377,NaN,NaN,left_only
720,5957,39.0,0,0,13577,NaN,NaN,left_only
743,6100,12.9,0,0,15529,NaN,NaN,left_only


C'est normal si onsale_web = 0 le fichier n'est pas en vente sur le net
verification si tous les NaN dans SKu sont bien dans les on_saleweb 0


,product_id,price,onsale_web,stock_quantity,sku,total_sales,post_name


Vérification doublons dans la base finale


,product_id,price,onsale_web,stock_quantity,sku,total_sales,post_name


### Exportation  vers fichier csv pour analyse 

In [9]:
# exportation vers fichier csv dans le dossier donnee_analyse
join_erp_venteweb.to_csv('donnee_analyse/analyse.csv',index=False)

In [10]:
help(perso)

Help on module fonction.fonction_perso in fonction:

NAME
    fonction.fonction_perso

FUNCTIONS
    analyseDataFrame(df)
        affiche un resumé des Dataframes info describe head et verification doublons
        
        Args:
            df : dataframe à analyser as dataFrame
        
        Returns:
            None
    
    categoriser(data, nom_col_recherche, mot_cle, nom_categorie, nom_col_categorie='categorie', regex=False)
        recherche un mot dans une string et enregistre un nom de categorie dans la colonne categorie (par defaut)
        Args:
            data : dataframe a categoriser as DataFrame
            nom_col_recherche : colonne où rechercher as string
            nom_categorie : nom de la catégorie as string
            nom_col_categorie : nom de la colonne cat"egorie a créer as string
        Returns :
            None
    
    courbe_Lorentz(plt_ax, df, colonne, titre='', titreX='', titreY='')
        configure une courbe de Lorentz sur l'axe matplotlib
    

In [11]:
pd.__version__

'1.4.3'